In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg

# Load the mock CSV data
df = pd.read_csv("mock_survey_data.csv")

# Cronbach's Alpha for 'support' items
support = df[['support_1', 'support_2', 'support_3']]
alpha, _ = pg.cronbach_alpha(support)
print("Support Cronbach’s Alpha:", round(alpha, 3))

# Compute averages for correlation
df['support_avg'] = support.mean(axis=1)
df['micro_avg'] = df[['micro_1', 'micro_2', 'micro_3']].mean(axis=1)
df['validation_avg'] = df[['validation_1', 'validation_2', 'validation_3']].mean(axis=1)

# Correlation matrix
corr = df[['support_avg', 'micro_avg', 'validation_avg', 'satisfaction', 'performance']].corr(method='spearman')
print("\nSpearman Correlation Matrix:\n", corr)

# Heatmap visualization
sns.heatmap(corr, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Spearman Correlation Heatmap")
plt.show()
